In [ ]:
!pip install transformers
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

train_df = pd.read_csv('non_skewed_train.csv')
test_df = pd.read_csv('non_skewed_test.csv')
df = pd.concat([train_df, test_df])
df

,id,text,label
0,86426.0,<user> ask native americans take,1
1,90194.0,<user> <user> go home drunk <user> maga trump2...,1
2,16820.0,amazon investigating chinese employees selling...,0
3,62688.0,<user> someone vetaken piece shit volcano <emo...,1
4,97670.0,<user> liberals kookoo,1
...,...,...,...
3531,NaN,eagles fucking trash,1
3532,78174.0,<user> <user> fraud amp fight anti semitism ex...,1
3533,95459.0,<user> thank jourdan amp hayley <emoji : face_...,0
3534,64878.0,<user> <user> fake news term used rather word ...,0


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

,id,text,label
0,67009.0,<user> think choice ppl stay hooked shit,1
1,NaN,rt <user> logic thinking nikki bad bitch,1
2,63714.0,<user> glad one watches,0
3,48529.0,<user> <user> <user> alaska girl chiming murko...,0
4,97052.0,<user> omg fuck lives sis,1
...,...,...,...
17675,93056.0,<user> <user> delirious,1
17676,62953.0,<user> <user> many arrests taken place sociali...,1
17677,47874.0,<user> <user> <user> rabid nationalists misled...,0
17678,38491.0,<user> looks like jokes liberals forttrump pol...,0


In [ ]:
test_df = df.sample(frac = 0.2, random_state = 42)
train_df = df.drop(test_df.index)

In [ ]:
train_df

,id,text,label
1,NaN,rt <user> 8220 <user> use buy 2 tees n wear de...,1
2,NaN,rt <user> shermans bark bite fuck bitch,1
4,67379.0,<user> <user> lmfao <emoji : rolling_on_the_fl...,0
6,87438.0,<user> manga reading,0
7,22717.0,<user> <user> <user> <user> <user> could kille...,0
...,...,...,...
17674,NaN,<user> lol yeah birds chirping outside bastards,1
17675,42814.0,<user> <user> <user> liberals mean conservatives,0
17676,75791.0,<user> get inner cities e chicago policies pro...,1
17677,70621.0,<user> person also needs change remainer put c...,0


In [ ]:
train_df.to_csv('non_skewed_train.csv', index = False)
test_df.to_csv('non_skewed_test.csv', index = False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'non_skewed_train.csv', 'test': 'non_skewed_test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 14144
    })
    test: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 3536
    })
})

In [ ]:
dataset = dataset.remove_columns([' id'])

In [ ]:
dataset['train'][0]

{'text': 'rt <user> 8220 <user> use buy 2 tees n wear dem bitches time 8221 lame ass lol',
 'label': 1}

In [ ]:
dataset['train'].features.keys()

dict_keys(['text', 'label'])

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 30)
  return encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14144 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

In [ ]:
train_ds = encoded_dataset["train"].shuffle(seed=42)
val_ds = encoded_dataset["test"].shuffle(seed=42)

In [ ]:
train_ds.set_format("torch")
val_ds.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

'0.27.2'

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bertbase-olid",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,  # Adjusted batch size
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=2,  # Gradient accumulation
    logging_steps=100,  # Adjust as needed
    eval_steps=200,  # Adjust as needed
)

In [ ]:
!pip install evaluate -q

In [ ]:
from transformers import AutoTokenizer, AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
scheduler = get_scheduler(
    "linear",
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_ds) // args.gradient_accumulation_steps * args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
      logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(predictions, labels)
    f1 = f1_score(predictions, labels, average = 'macro')

    return {"accuracy": acc, "f1": f1}

In [ ]:
train_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 43516
})

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.371700,0.343915,0.848416,0.848342
2,0.342600,0.419486,0.820419,0.818850
3,0.268400,0.442856,0.843043,0.842990
4,0.186100,0.688717,0.830317,0.830299
5,0.119400,0.759969,0.825226,0.825097


TrainOutput(global_step=4420, training_loss=0.2515983099311725, metrics={'train_runtime': 678.087, 'train_samples_per_second': 104.293, 'train_steps_per_second': 6.518, 'total_flos': 1090266435648000.0, 'train_loss': 0.2515983099311725, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(val_ds)

In [ ]:
val_ds[0]

{'text': 'guess night bitches die stewie nigga',
 'label': 1,
 'input_ids': [101,
  3984,
  2305,
  7743,
  2229,
  3280,
  20717,
  2666,
  9152,
  23033,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
preds

PredictionOutput(predictions=array([[-2.480654  ,  2.8042836 ],
       [ 1.7039399 , -0.78020996],
       [ 0.26686358,  0.42784414],
       ...,
       [ 0.5593707 ,  0.29479125],
       [ 1.6615864 , -0.7452464 ],
       [ 0.6045313 ,  0.321739  ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.34391507506370544, 'test_accuracy': 0.8484162895927602, 'test_f1': 0.8483424943099309, 'test_runtime': 7.3385, 'test_samples_per_second': 481.844, 'test_steps_per_second': 60.231})

In [ ]:
pred_vals = np.argmax(preds.predictions, axis=-1)
labels = preds.label_ids
pred_vals, labels

(array([1, 0, 1, ..., 0, 0, 0]), array([1, 0, 0, ..., 0, 0, 0]))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
print('roc: ', roc_auc_score(labels, pred_vals))
print('f1: ', f1_score(labels, pred_vals, average = 'macro'))
print('accuracy: ', accuracy_score(labels, pred_vals))

roc:  0.8490937019969278
f1:  0.8483424943099309
accuracy:  0.8484162895927602


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
trainer.push_to_hub('base-bert-olid')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1709646340.6754295aefd8.984.0:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ARC4N3/bertbase-olid/commit/083a6eb555c0888bb7205abbccbd181e5c65bc7f', commit_message='base-bert-olid', commit_description='', oid='083a6eb555c0888bb7205abbccbd181e5c65bc7f', pr_url=None, pr_revision=None, pr_num=None)